### Análise de eleição - Eduardo Paes (55)

In [24]:
import sqlite3
import sql_utils
import pandas as pd

In [2]:
conn = sqlite3.connect(database='databases/eleicoes_2024_rj.sql')
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'databases/eleicoes_2024.sql' AS geral")
cursor.execute("ATTACH DATABASE 'databases/eleicoes_2024_rj.sql' AS rj")

In [70]:
query_join = """
    SELECT *
    FROM rj.resultados
    INNER JOIN geral.locais
        ON rj.resultados.NR_LOCAL_VOTACAO = geral.locais.NR_LOCAL_VOTACAO
        AND rj.resultados.SG_UF = geral.locais.SG_UF
        AND rj.resultados.CD_MUNICIPIO = geral.locais.CD_MUNICIPIO
    WHERE NM_VOTAVEL LIKE '%EDUARDO%PAES%';
"""
query_find = """
    SELECT *
    FROM rj.resultados
    WHERE NM_VOTAVEL LIKE '%EDUARDO%PAES%';
"""


- Nome: NR_LOCAL_VOTACAO, Tipo: INTEGER
- Nome: SG_UF, Tipo: TEXT
- Nome: CD_MUNICIPIO, Tipo: INTEGER

In [71]:
result = pd.read_sql_query(query_join, conn)

In [72]:
result

,NR_LOCAL_VOTACAO,SG_UF,CD_MUNICIPIO,ANO_ELEICAO,NR_TURNO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NM_BAIRRO,NR_LATITUDE,NR_LONGITUDE,QT_ELEITOR_SECAO
0,1317,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,170,1317,RJ,60011,RIO DE JANEIRO,BENFICA,-22.892385,-43.234194,460
1,1317,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,170,1317,RJ,60011,RIO DE JANEIRO,BENFICA,-22.892385,-43.234194,465
2,1317,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,170,1317,RJ,60011,RIO DE JANEIRO,BENFICA,-22.892385,-43.234194,468
3,1317,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,170,1317,RJ,60011,RIO DE JANEIRO,BENFICA,-22.892385,-43.234194,469
4,1317,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,170,1317,RJ,60011,RIO DE JANEIRO,BENFICA,-22.892385,-43.234194,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090297,1210,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,47,1210,RJ,60011,RIO DE JANEIRO,TURIAÇÚ,-22.860852,-43.344631,482
2090298,1210,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,47,1210,RJ,60011,RIO DE JANEIRO,TURIAÇÚ,-22.860852,-43.344631,483
2090299,1210,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,47,1210,RJ,60011,RIO DE JANEIRO,TURIAÇÚ,-22.860852,-43.344631,483
2090300,1210,RJ,60011,2024,1,11,Prefeito,55,EDUARDO DA COSTA PAES,47,1210,RJ,60011,RIO DE JANEIRO,TURIAÇÚ,-22.860852,-43.344631,484
